# Sinkhorn iterations
https://www.youtube.com/watch?v=BfOjrQAhG4M

**Disclaimer**. Все написанное ниже -- результаты чтения кода и запуска `log_optimal_transport` на пробных данных + моя интерпретация. Я не нашел хороших комментов или релевантного текста в статье. Не исключено, что тут не все верно написано, стоит читать и проверять внимательно, особенно всякие индексы-оси и.т.д.


## Формулировка
Пусть даны два набора ключевых точек (kp): набор $n$ размера N (например синий), и набор $m$ размера M (например красный).

Для данной матрицы схожести ключевых точек $ S_{ij}[N, M] = (f^n_{i}, f^m_{j}) $ и данного параметра $\alpha_d$ (dustbin) надо найти матрицу неотрицательных $P_{ij}[N+1, M+1]$ такую что:
1. максимальна $ C = \sum_{i,j}^{N+1, M+1} P_{ij} \hat{S_{ij}}$, где $\hat{S}[N+1, M+1]$ (coupling в коде) - это $S[N, M]$ расширенная(конкатенированная) строкой и столбцом, заполненными значением $\alpha_d$  
1. для любых i <= N выполняется: $\sum_{j}^{M+1}P_{ij} = 1$ (каждый i-й(синий) kp из $n$ "перетекает" с коэффициентами $P_{ij}$ в j-е kp из $m$ и в m-dustbin)
1. для любых j <= M выполняется: $\sum_{i}^{N+1}P_{ij} = 1$ (каждый j-й(красный) kp из $m$ "перетекает" с коэффициентами $P_{ij}$ в i-е kp из $n$ и в n-dustbin)



## Расширенная формулировка с учетом dustbin
Пункты выше - стандартные для оптимального транспорта, но у нас есть еще dustbin-ы, которые надо учесть отдельно. Для реальных kp ограничения по перетеканию строгие - т.е. сумма вероятностей должна быть **равна** 1. Для dustbin ограничения нестрогие по смыслу : в m-dustbin может попасть от 0 до N kp, в n-dustbin может попасть от 0 до M kp. Проблема в том, что метод множителей Лагранжа предполагает, что ограничения заданы строгими равенствами (хотя вообще есть расширения метода на неравенства). Поэтому авторы, судя по коду, делают такой трюк - они задают следующие **строгие** ограничения для каждого dustbin:
1. $\sum_{i}^{N+1}P_{i,M+1} = N$
1. $\sum_{j}^{N+1}P_{N+1,j} = M$

На первый взгляд кажется, что это какая-то фигня: получается что N синих kp и M красных kp должны быть отправлены в соответствующие dustbin, т.е. вообще все ключевые точки должны быть отправлены в dustbin. Но так не происходит из за последнего (правого углового) элемента матрицы $P_{N+1,M+1}$, который по смыслу не соответствует ни матчингу какой-то реальной точки с другой точкой, ни матчингу реальной точки с dustbin. Этот элемент участвует только в нормировке dustbin, потому может принимать любое значение - можно считать его численной заглушкой. После итераций его значение всегда получается равным сумме вероятностей всех сматченных точек. Можно визуализировать получающееся решение после итераций такой блок-матрицей (указаны суммы значений элементов блоков, а не сами элементы; KP - ключевые точки, DB - dustbin):

KP | Kp N        || DB
---|-------------||---
KpM|     P       || M - P 
   |             ||
DB |  N - P      || P

Сумма всех реальных сматченных точек = P, это примерно количество матчей. Сумма всех элементов матрицы = M+N **всегда** (для любого P) - это количество kp. Сумма (количество) всех реальных несматченных точек - (N-P) и (M-P) соответственно, что логично. Сумма каждого из dustbin N и M соответственно. Вся "структура" матрицы определяется всего одной переменной P(количество матчей) - поэтому матрица-решение такого вида существует всегда. Например:
1. Если дано N = M точек, и все сматчены, то P=N=M (количество матчей), а N-P=M-P=0 (количество отправленных в dustbin)
1. Если даны N и M точек, и никто ни с кем не с матчен, то P=0, N-P=N, M-P=M

В итоге довольно странные ограничения на dustbin приводят к логичному решению с более-менее интерпретируемыми свойствами. Интересно, как до этого додумались авторы, имхо это все ни разу не очевидно.



## [Лагранжиан](https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%BC%D0%BD%D0%BE%D0%B6%D0%B8%D1%82%D0%B5%D0%BB%D0%B5%D0%B9_%D0%9B%D0%B0%D0%B3%D1%80%D0%B0%D0%BD%D0%B6%D0%B0)
Лагранжиан состоит из четырех частей:
1. Максимизируемая сумма "схожести"
1. Добавка из Sinkhorn Iterations $P \cdot log(P)$. Возьмем ее с коэффициентом 1 для простоты, могли бы умножить на 0.1 или 0.42 - влияет на "сглаженность" оптимального решения.
1. Ограничения реальных матчей: "по i" (для синих), множители Лагранжа $l^{u}$, в коде $u = l^u$, "по j" (для красных), множители Лагранжа $l^{v}$, в коде $v = l^v$
1. Ограничения для dustbin (записываем их сначала отдельно для лучшей читаемости). Множители лагранжа для них хранятся как последний элемент $l^{u}_{N+1}$ и $l^{v}_{M+1}$, аналогично тому как сделано в коде.  

Получается:
$$ L(P_{ij}[N+1, M+1], l^{u}_{i}[N+1], l^{v}_{j}[M+1]) = $$
$$\sum_{i,j}^{N+1, M+1} P_{ij} \hat{S_{ij}} +   
\sum_{i,j}^{N+1, M+1} P_{ij} (log(P_{ij}) - 1) + \newline
\sum_{i}^{N} l^{u}_{i} (\sum_{j}^{M+1}P_{ij} - 1) + \sum_{j}^{M} l^{v}_{j} (\sum_{i}^{N+1}P_{ij} - 1) + \newline
l^{u}_{N+1} (\sum_{j}^{M+1}P_{N+1,j} - M) + l^{v}_{M+1} (\sum_{i}^{N+1}P_{i,M+1} - N)
$$ 

Для удобства запишем две последних части в одну: обозначим как $\mu[N+1]$ и $\nu[M+1]$ требуемые значения сумм строк и столбцов $P_{ij}$ - это 1 везде, кроме последних N+1 и M+1 элементов - там стоят M и N соответственно. Также перенесем множители Лагранжа под суммирование по i, j. Тогда запись чуть короче будет:

$$ L(P_{ij}[N+1, M+1], l^{u}_{i}[N+1], l^{v}_{j}[M+1]) = \newline \sum_{i,j}^{N+1, M+1}  
 [ P_{ij} \hat{S_{ij}} + P_{ij} (log(P_{ij}) - 1) + l^{u}_{i} (P_{ij} - \mu_{i}) + l^{v}_{j} (P_{ij} - \nu_{j}) ]
$$ 


## Вывод уравнений для итераций
#### Производная по $P_{ij}$
Необходимым условием экстремума является равенство нулю каждой производной $dL/P_{ij}$, а также равенства нулю производных $dL/dl^{u}_{i}$, $dL/dl^{v}_{j}$.
Производные по множителям Лагранжа дадут снова исходные ограничения c $\mu$ и $\nu$.

Производная по $P_{ij}$:
$$
dL/dP_{ij} = [ \hat{S_{ij}} + (log(P_{ij}) - 1) + P_{ij} / P_{ij} + l^{u}_{i} + l^{v}_{j} ] = \newline
 = \hat{S_{ij}} + log(P_{ij}) + l^{u}_{i} + l^{v}_{j}
$$
Приравниваем 0, выражаем log(P)

$$
log(P_{ij}) = -\hat{S_{ij}} - l^{u}_{i} - l^{v}_{j}
$$
В таком решении получилось, что чем **больше** $S_{ij}$, тем **меньше** $P_{ij}$, что нелогично - мы же максимизируем сумму, если две точки схожи по S - вероятность матча высокая. Метод Лагранжа ищет не максимум или минимум, а все точки экстремума, которые не меняются, если домножить на -1. Обычно этого не делается, т.к. решается задача минимизации стоимости, а не максимизации схожести.

Чтобы решение имело более "интуитивный" смысл - домножим (мысленно) в исходном лагранжиане первую часть на -1, тогда получится почти такой же результат с точностью до знака при S.
Для множителей Лагранжа знак не имеет значения (с точностью до обозначения это одно и то же). Чтобы получить те же знаки, что и у авторов в коде - "инвертируем"(домножим на -1) и их тоже. 
$$
log(P_{ij}) = \hat{S_{ij}} + l^{u}_{i} + l^{v}_{j}
$$

В довесок еще обозначим $ Z_{ij} = log(P_{ij}) $ (т.е. $P_{ij} = exp(Z_{ij})$). Используем такую замену (по анологии с кодом) по двум причинам:
1. В логарифмическом представлении стабильнее вычисления
1. В коде в качестве лосса используется кросс-энтропия, которая принимает на вход не вероятности (значения от 0 до 1), а **логиты**(значения от -inf до 0) 

<font color='#dd33dd'>
$$
Z_{ij} =  log(P_{ij}) = \hat{S_{ij}} + l^{u}_{i} + l^{v}_{j}
$$
</font> 

#### Производная по $l^u, l^v$
Производные по $l^u, l^v$ тоже должны быть равны 0, они просто дают исходные ограничения (при вычислении надо правильно учесть внешнюю сумму, лучше видно из длинной записи Лагранжиана):

$dL/dl^{u}_i = \sum_{j}^{M+1}P_{ij} - \mu_i = 0$

$dL/dl^{v}_j = \sum_{i}^{N+1}P_{ij} - \nu_i = 0$


#### Log-space
Прологарифмируем $dL/dl^{u}_i$, и используем замену $P_{ij} = exp(Z_{ij}) $.

$$
log ( \sum_{j}^{M+1}exp(Z_{ij})) = log(\mu_i) \newline
$$

$$
log ( \sum_{j}^{M+1}exp(\hat{S_{ij}} + l^{u}_{i} + l^{v}_{j})) = log(\mu_i) \newline
$$

$$
log ( \sum_{j}^{M+1}exp(\hat{S_{ij}})\cdot exp(l^{u}_{i})\cdot exp(l^{v}_{j}))) = log(\mu_i)
$$

Суммирование здесь только по j, часть с i можно вынести из под суммы:
$$
log (exp(l^{u}_{i}) \sum_{j}^{M+1}exp(\hat{S_{ij}})\cdot exp(l^{v}_{j}))) = log(\mu_i)
$$
По свойству логарифма log(ab) = log(a) + log(b), log(exp(x)) = x:
$$
l^{u}_{i} + log(\sum_{j}^{M+1}exp(\hat{S_{ij}})\cdot exp(l^{v}_{j}))) = log(\mu_i) \newline
$$


<font color='#33dd33'>
$$
l^{u}_{i} = log(\mu_i) - log(\sum_{j}^{M+1}exp(\hat{S_{ij}} + l^{v}_{j}))) 
$$
</font> 
Точно также для $l^{v}_{j}$:

<font color='#33dd33'>
$$
l^{v}_{j} = log(\nu_j) - log(\sum_{i}^{N+1}exp(\hat{S_{ij}} + l^{u}_{i}))) 
$$
</font> 

# Зеленые уравнения выше - Sinkhorn Iterations в log-space!

In [3]:
import torch
"""
Вызов в модуле:

        scores = torch.einsum('bdn,bdm->bnm', mdesc0, mdesc1)
        scores = scores / self.config['descriptor_dim']**.5

        # Run the optimal transport.
        scores = log_optimal_transport(
            scores, self.bin_score,
            iters=self.config['sinkhorn_iterations'])
"""

def log_optimal_transport(scores, alpha, iters: int):
    """ Perform Differentiable Optimal Transport in Log-space for stability"""
    # scores - матрица S[B, N, M]
    b, m, n = scores.shape # [B, N, M]
    one = scores.new_tensor(1) # scalar 1
    ms, ns = (m*one).to(scores), (n*one).to(scores) # scalar N, scalar M 
    # dustbins: alpha x shape
    bins0 = alpha.expand(b, m, 1) 
    bins1 = alpha.expand(b, 1, n)
    alpha = alpha.expand(b, 1, 1)
    
    # [B, N + 1, M + 1], она же S с крышкой
    couplings = torch.cat([torch.cat([scores, bins0], -1),
                           torch.cat([bins1, alpha], -1)], 1)
    
    # Как я понял, здесь norm - это еще один трюк для стабилизации.
    # norm = -log(M+N), и сначала он добавляется в log_mu, log_nu,
    # а потом вычитается из Z.
    # Я протестировал, что если заменить norm на log(1), т.е. на 0
    # то все продолжает работать
    norm = - (ms + ns).log() # можно заменить на one.log()
    
    # [ -log(M+N) ... x M ... -log(M+N), log(M) - log(M+N)] [M+1] 
    # В последний элемент прибавляется log(M) или log(N)
    # Если norm=log(1) = 0, то вектор равен
    # [log(1), log(1) ..., log(M)]
    log_mu = torch.cat([norm.expand(m), ns.log()[None] + norm])
    log_nu = torch.cat([norm.expand(n), ms.log()[None] + norm])
        
    # бродкаст на все батчи
    log_mu, log_nu = log_mu[None].expand(b, -1), log_nu[None].expand(b, -1)
    # выполняем iters итераций, получаем логарифм(!) вероятности матча
    Z = log_sinkhorn_iterations(couplings, log_mu, log_nu, iters)
    # откатываем norm обратно (вычитаем -log(M+N))
    Z = Z - norm  # multiply probabilities by M+N
    return Z

def log_sinkhorn_iterations(Z, log_mu, log_nu, iters: int):
    """ Perform Sinkhorn Normalization in Log-space for stability"""
    # инициализируем нулями u,v множители Лагранжа
    u, v = torch.zeros_like(log_mu), torch.zeros_like(log_nu)
    for _ in range(iters):
        # torch.logsumexp = torch.log(torch.sum(torch.exp(...)))
        u = log_mu - torch.logsumexp(Z + v.unsqueeze(1), dim=2)
        v = log_nu - torch.logsumexp(Z + u.unsqueeze(2), dim=1)
    return Z + u.unsqueeze(2) + v.unsqueeze(1)

## Тесты

In [12]:
t = (torch.randn((1, 3, 5)) - 0.5) * 2
t

tensor([[[-0.3978,  2.0107, -2.6327,  1.1628, -0.7444],
         [-1.6935, -1.7384, -2.9961, -4.4853, -0.2840],
         [-1.1089, -3.1430, -1.7885, -0.7323,  0.0943]]])

In [22]:
a = torch.Tensor([0.00042])

In [23]:
torch.round(torch.exp(log_optimal_transport(t, a, 2)) * 100).numpy()


array([[[  6.,  44.,   1.,  24.,   3.,  18.],
        [  7.,   4.,   2.,   0.,  19.,  71.],
        [  9.,   1.,   5.,  11.,  22.,  55.],
        [ 78.,  51.,  92.,  65.,  56., 156.]]], dtype=float32)

In [5]:
# сумма по 1 оси
# Сумма dustbin = N
torch.exp( # Матрица возводится в экспоненту!
    log_optimal_transport(t, a, 2)
).numpy().round(2).sum(axis=1)

array([[1.  , 1.01, 0.99, 1.  , 1.  , 3.  ]], dtype=float32)

In [95]:
# сумма по 2 оси
# Сумма dustbin = M
(torch.exp(log_optimal_transport(t, a, 2))).numpy().round(2).sum(axis=2)

array([[0.98      , 0.97999996, 0.8       , 5.25      ]], dtype=float32)

In [96]:
# Сумма всей матрицы
# M + N
(torch.exp(log_optimal_transport(t, a, 2))).numpy().round(2).sum()

8.01

In [97]:
# Результаты при 1 итерации
torch.exp(log_optimal_transport(t, a, 1)).numpy().round(2)

array([[[0.  , 0.  , 0.04, 0.07, 0.47, 0.21],
        [0.06, 0.35, 0.  , 0.17, 0.  , 0.25],
        [0.  , 0.  , 0.02, 0.41, 0.01, 0.09],
        [0.94, 0.64, 0.94, 0.36, 0.53, 2.45]]], dtype=float32)

In [98]:
# Результаты при 3 итерациях (разница большая с iters=1)
torch.exp(log_optimal_transport(t, a, 3)).numpy().round(2)

array([[[0.  , 0.01, 0.06, 0.06, 0.58, 0.3 ],
        [0.09, 0.45, 0.  , 0.13, 0.  , 0.33],
        [0.  , 0.  , 0.05, 0.62, 0.01, 0.23],
        [0.91, 0.55, 0.89, 0.19, 0.41, 2.14]]], dtype=float32)

In [100]:
# Результаты при 10 итерациях (разница не оч большая с iters=3)
torch.exp(log_optimal_transport(t, a, 10)).numpy().round(2)

array([[[0.  , 0.01, 0.06, 0.05, 0.58, 0.3 ],
        [0.09, 0.45, 0.  , 0.12, 0.  , 0.33],
        [0.  , 0.  , 0.06, 0.66, 0.01, 0.27],
        [0.91, 0.54, 0.88, 0.17, 0.4 , 2.1 ]]], dtype=float32)